<a href="https://colab.research.google.com/github/koperak/Advent-of-Code-2024/blob/main/AoC_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialize support

In [1]:
import os
from pathlib import Path
from shutil import copyfile
import requests
from datetime import date
import logging
from argparse import ArgumentParser

logging.basicConfig(
    level=logging.INFO, format="%(levelname)-8s %(funcName)s: %(message)s",
)
logger = logging.getLogger(__name__)


BASE_DIR = "sample_data"
TEMPLATES_DIR = BASE_DIR + "/templates"


def setup_dir(day):
  newdir = Path(f"day{day:02d}")
  if not newdir.exists():
      os.mkdir(newdir)
      logger.info(f"created a new directory {newdir}")
  logger.info(f"done copying templates to {newdir}")


def get_input(day, year):

  from google.colab import userdata

  cookie = userdata.get('AOC_SESSION')

  if not cookie:
      logger.warning("no environment variable 'AOC_SESSION' found! skipping download")
      return

  url = f"https://adventofcode.com/{year}/day/{day}/input"
  dayfolder = f"day{day:02d}"
  if "input.txt" in os.listdir(dayfolder):
      logger.warning(f"{dayfolder}/input.txt already exists, skip download")
      return

  logger.info(f"download input from {url}... ")
  try:
      response = requests.get(
          url=url,
          cookies={"session": cookie},
          headers={"User-Agent": "https://github.com/davekch/aoc by dave-koch@web.de"},
      )
      if response.ok:
          data = response.text
          f = open(Path(dayfolder) / "input.txt", "w+")
          f.write(data.rstrip("\n"))
          f.close()
          logger.info("... done!")
      else:
          logger.error("server response not ok")
  except:
      logger.error("something went wrong")


Day 1 year 2024

Setup test input and test data

In [2]:
setup_dir(1)
get_input(1,2024)
input = open("day01/input.txt").read().splitlines()
test_input = """3   4
4   3
2   5
1   3
3   9
3   3"""
test_input = test_input.split("\n")

In [3]:
def prepare_input(input):
# prepare input
# return left and right list of integers
  _list_a, _list_b = zip(*[map(int, row.split("   ")) for row in input])
  return _list_a, _list_b

Part 1

In [4]:
left_list, right_list  = prepare_input(input)

In [5]:
sum(map(lambda x,y: abs(x - y), sorted(left_list), sorted(right_list)))

1258579

Part 2

In [6]:
from collections import OrderedDict, Counter
similarity_scores = Counter(right_list)

In [7]:
sum(map(lambda first_number: first_number * similarity_scores.get(first_number,0), left_list))

23981443

# Day 2 year 2024

## Setup test input and test data

In [8]:
setup_dir(2)
get_input(2,2024)
input = open("day02/input.txt").read().splitlines()
test_input = """7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9"""
test_input = test_input.split("\n")

In [9]:
def prepare_input(input):
# prepare input
# return left and right list of integers
  _list = [list(map(int, row.split())) for row in input]
  return _list

In [10]:
reports  = prepare_input(input)

## Part 1

In [11]:
differences = [[y - x for x, y in zip(report, report[1:])] for report in reports]

In [12]:
def is_increasing_or_decreasing_and_between2_3(difference):
  is_increasing = all(diff > 0 for diff in difference)
  is_decreasing = all(diff < 0 for diff in difference)
  is_between2_3 = all(1 <= abs(diff) <= 3 for diff in difference)
  return (is_increasing or is_decreasing) and is_between2_3

In [13]:
safe = 0
for difference in differences:
  if is_increasing_or_decreasing_and_between2_3(difference):
    safe = safe + 1

print(f'Part 1: {safe}')

Part 1: 257


##Part 2

In [14]:
def remove_one(input_list):
  """ input_list: list of integers
      retuns: yelds evety combinatnion of input_list with one element removed
  """
  for i in range(len(input_list)):
    yield input_list[:i] + input_list[i+1:]

In [15]:
def calc_differences(input_list):
  return [y - x for x, y in zip(input_list, input_list[1:])]

In [16]:
safe = 0
for report in reports:
  difference = calc_differences(report)
  if is_increasing_or_decreasing_and_between2_3(difference):
    safe = safe + 1
  else:
    for one_removed in remove_one(report):
      difference = calc_differences(one_removed)
      if is_increasing_or_decreasing_and_between2_3(difference):
        safe = safe + 1
        break

print(f'Part 2: {safe}')

Part 2: 328


# Day 3 year 2024

## Setup test input and test data

In [17]:
day = 3
setup_dir(day)
get_input(day,2024)
input = open(f"day0{day}/input.txt").read()
test_input = """xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"""

In [18]:
def prepare_input(input):
# prepare input
# return left and right list of integers
  return input

In [19]:
memory  = prepare_input(input)

## Part 1

In [20]:
import re

pattern = r'mul\((\d{1,3}),(\d{1,3})\)'

print(sum(x * y for x, y in [map(int, mul) for mul in re.findall(pattern, input)]))



180233229


##Part 2

In [21]:
new_pattern =  r"mul\(\d+,\d+\)|do\(\)|don't\(\)"

test_input = """xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"""
memory = prepare_input(input)

suma = 0
follow = True

for found in re.findall(new_pattern, input):
  match found:
      case "do()":
          follow = True
      case "don't()":
          follow = False
      case _:
          if follow:
              x, y = map(int, found[4:-1].split(','))
              suma += x * y

print(suma)

95411583


# Day 4 year 2024

## Setup test input and test data

In [22]:
day = 4
setup_dir(day)
get_input(day,2024)
input = open(f"day0{day}/input.txt").read()
test_input = """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""

In [23]:
def prepare_input(input):
  return input.split()

In [24]:
words = prepare_input(input)

## Part 1

In [25]:
directions = [(0, 1), (0, -1), (-1, 0), (1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]

In [26]:
def validate(row_pos, col_pos, x_dir, y_dir, word_matrix, word):
  """ validate if whole word can fit from location (row_pos, col_pos) into direction (x_dir, y_dir)"""

  rows_num = len(word_matrix)
  cols_num = len(word_matrix[0])

  for i, letter in enumerate(word):
      x_n = row_pos + x_dir * i
      y_n = col_pos + y_dir * i
      if x_n < 0 or y_n < 0 or x_n >= rows_num or y_n >= cols_num or word_matrix[x_n][y_n] != letter:
          return False
  return True

In [27]:
rows = len(words)
cols = len(words[0])
xmas = "XMAS"

In [28]:
count = 0

for row in range(rows):
     for column in range(cols):
          for dirx, diry in directions:
               if validate(row, column, dirx, diry, words, xmas):
                    count += 1

print(count)

2483


##Part 2

In [31]:
rows = len(words)
cols = len(words[0])
count = 0

for row in range(1, rows - 1):
  for column in range(1, cols - 1):
    skos1 = words[row - 1][column - 1] + words[row][column] + words[row + 1][column + 1] # skos \
    skos2 = words[row - 1][column + 1] + words[row][column] + words[row + 1][column - 1] # skos /
    if (skos1 == "MAS" or skos1 == "SAM") and (skos2 == "MAS" or skos2 == "SAM"):
      count += 1

print(count)

1925


# Day 5 year 2024

## Setup test input and test data

In [47]:
day = 5
setup_dir(day)
get_input(day,2024)
input = open(f"day0{day}/input.txt").read()
test_input = """47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47"""

In [48]:
def prepare_input(input):
  # Split input into rules and updates
  rules_section, updates_section = input.strip().split("\n\n")
  rules = [tuple(map(int, line.split("|"))) for line in rules_section.splitlines()]
  updates = [list(map(int, line.split(","))) for line in updates_section.splitlines()]
  return rules, updates

In [49]:
rules, updates = prepare_input(input)

## Part 1

In [53]:
from functools import cmp_to_key

def comparator(x, y, rules):
  # If there's a rule x|y, x should come before y
  if (x, y) in rules:
      return -1
  if (y, x) in rules:
      return 1
  return 0  # No direct rule between x and y

def find_middle_page(update):
  return update[len(update) // 2]

def sum_middle_pages(rules, updates):
  total = 0

  for update in updates:
      sorted_update = sorted(update, key=cmp_to_key(lambda x, y: comparator(x, y, rules)))
      if sorted_update == update:
          total += find_middle_page(update)

  return total

In [54]:
result = sum_middle_pages(rules, updates)
print(f"Sum of middle pages: {result}")

Sum of middle pages: 7024


##Part 2